In [1]:
import nltk
import pandas as pd
import numpy as np

In [2]:
data_pos = []
data_neg = []
with open("./../train_pos.txt") as f:
    for i in f: 
        t = i.replace('<user>', '')
        t1 = t.replace('<url>', '')
        data_pos.append(t1)

with open("./../train_neg.txt") as f:
    for i in f:
        t = i.replace('<user>', '')
        t1 = t.replace('<url>', '')
        data_neg.append(t1)

In [3]:
data_pos_ls = [tweet.split(" ") for tweet in data_pos]
data_neg_ls = [tweet.split(" ") for tweet in data_neg]
pd_tweet_pos = pd.Series(data_pos_ls)
pd_tweet_neg = pd.Series(data_neg_ls)

In [52]:
print(pd_tweet_pos.shape)
print(pd_tweet_neg.shape)

(100000,)
(100000,)


dataset is already tokenized so that step is not needed
next step would be lemmatizer

In [12]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abhinavkumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/abhinavkumar/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhinavkumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/abhinavkumar/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [53]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [54]:
pos_tag(pd_tweet_pos[9])

[('this', 'DT'),
 ('is', 'VBZ'),
 ('were', 'VBD'),
 ('i', 'JJ'),
 ('belong\n', 'NN')]

In [16]:
tmp = lemmatize_sentence(pd_tweet_pos[9])

In [55]:
print(tmp)
print(pd_tweet_pos[9])

['this', 'be', 'be', 'i', 'belong\n']
['this', 'is', 'were', 'i', 'belong\n']


remove noise from the dataset

remove hyperlinks, punctuation and special characters

In [57]:
import re, string

personlised_remove_reg_tokens = ["\n", "..."]
personlised_remove_tokens = ["rt"]

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        

        """
        need to look into this
        """
        
        # for rm_token in personlised_remove_tokens:
        #     token = re.sub(rm_token,"", token)

        # for rm_token in personlised_remove_reg_tokens:
        #      pat = r'(\w*%s\w*)' % rm_token
        #      token = re.sub(pat,"", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [58]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in pd_tweet_pos:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in pd_tweet_neg:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))


### determine word density

for analysis purpose

In [33]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [34]:
all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [35]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[('!\n', 11330), ('\n', 10552), ('love', 9139), ('get', 8568), ('rt', 7263), ('.\n', 7166), ('follow', 7142), ("i'm", 6990), ('good', 6814), ('...', 6696)]


### Naive Bayes Classifiers NLTK

In [59]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [60]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [61]:
import random

positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset
print(len(dataset))

random.shuffle(dataset)

train_data = dataset[:150000]
test_data = dataset[150000:]

200000


In [62]:
print(len(train_data))
print(len(test_data))

150000
50000


In [63]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.75432


In [64]:
print(classifier.show_most_informative_features(20))

Most Informative Features
               paperback = True           Negati : Positi =    747.1 : 1.0
                  custom = True           Negati : Positi =    326.0 : 1.0
             replacement = True           Negati : Positi =    283.0 : 1.0
                      (8 = True           Negati : Positi =    263.6 : 1.0
             manufacture = True           Negati : Positi =    242.9 : 1.0
                   ounce = True           Negati : Positi =    236.8 : 1.0
                   frame = True           Negati : Positi =    148.4 : 1.0
                    misc = True           Negati : Positi =    143.3 : 1.0
                   cable = True           Negati : Positi =    138.5 : 1.0
                  walnut = True           Negati : Positi =    135.3 : 1.0
    #yougetmajorpointsif = True           Positi : Negati =    131.4 : 1.0
               stainless = True           Negati : Positi =    114.6 : 1.0
                 apparel = True           Negati : Positi =    109.3 : 1.0